In [133]:
import importlib

importlib.reload(toolbox.fasta)
importlib.reload(toolbox.hamming)
importlib.reload(toolbox.alignment)
importlib.reload(toolbox.pdb)
importlib.reload(toolbox.msa)

<module 'toolbox.msa' from '/Users/eugen/code/bioinformatics-toolbox/toolbox/msa.py'>

In [134]:
from os.path import join

DATA_FOLDER = "sample_data"

## FASTA

We can load the fasta file like this

In [135]:
import toolbox.fasta

sequences = toolbox.fasta.parse(join(DATA_FOLDER, "test.fasta"))
sequences = list(sequences)
seq = sequences[0]
seq

Sequence(description='sp|P83673|LYS1_CRAVI Lysozyme 1 OS=Crassostrea virginica OX=6565 GN=lysoz1 PE=1 SV=3', name='sp|P83673|LYS1_CRAVI', sequence='MNGLILFCAVVFATAVCTYGSDAPCLRAGGRCQHDSITCSGRYRTGLCSGGVRRRCCVPSSSNSGSFSTGMVSQQCLRCICNVESGCRPIGCHWDVNSDSCGYFQIKRAYWIDCGSPGGDWQTCANNLACSSRCVQAYMARYHRRSGCSNSCESFARIHNGGPRGCRNSNTEGYWRRVQAQGCN')

We can access the name of the sequence..


In [136]:
seq.name

'sp|P83673|LYS1_CRAVI'

In [137]:
...its description...

SyntaxError: invalid syntax (<ipython-input-137-8fb293af116a>, line 1)

In [138]:
seq.description

'sp|P83673|LYS1_CRAVI Lysozyme 1 OS=Crassostrea virginica OX=6565 GN=lysoz1 PE=1 SV=3'

...its content...

In [139]:
seq.sequence

'MNGLILFCAVVFATAVCTYGSDAPCLRAGGRCQHDSITCSGRYRTGLCSGGVRRRCCVPSSSNSGSFSTGMVSQQCLRCICNVESGCRPIGCHWDVNSDSCGYFQIKRAYWIDCGSPGGDWQTCANNLACSSRCVQAYMARYHRRSGCSNSCESFARIHNGGPRGCRNSNTEGYWRRVQAQGCN'

...and its length.

In [140]:
len(seq)

184

The `Sequence` is also an `Iterable` and a `Mapping`, so we can loop through it and access the residues by index.

In [141]:
for r in seq[10:15]:
    print(r.upper())

V
F
A
T
A


## Hamming distance

Let's try the hamming distance on simple strings first, so that we can check it's working correctly.

In [142]:
import toolbox.hamming

for a, b in [("ABCD", "CBDD"), ("HOKSZA", "HOKZSA"), ("WYBITUL", "WYTUBIL")]:
    print(f"The distance between {a} and {b} is {toolbox.hamming.distance (a, b)}")

The distance between ABCD and CBDD is 2
The distance between HOKSZA and HOKZSA is 2
The distance between WYBITUL and WYTUBIL is 4


Now, let's use the sequences we loaded above (notice how the function works for strings as well as sequences). Firstly the distance of a sequence to itself should be 0...

In [143]:
toolbox.hamming.distance(seq, seq)

0

Secondly, we can only measure distances between equally sized sequences. Let's pull up the other sequence from the fasta file, and compute its distance to the first one.

In [144]:
seq2 = sequences[1]
toolbox.hamming.distance(seq, seq2)

ValueError: The sequences should have identical lengths, but the lengths are 184, 607

## Alignment

The `align` function returns a dictionary with the Levenshtein edit distance and the alignment itself.

Let's start with simple strings again...

In [145]:
import toolbox.alignment

A = "clock"
B = "lacks"

res = toolbox.alignment.align(A, B)
res["distance"]

3

...and to view the alignment(s):

In [146]:
for s1, s2 in res["alignments"]:
    print("".join(s1))
    print("".join(s2))

clock-
-lacks


If there's more optimal alignments, we get all of them.

In [147]:
res = toolbox.alignment.align("ABBBD", "ABVD")
for s1, s2 in res["alignments"]:
    print("".join(s1))
    print("".join(s2))
    print()

ABBBD
ABV-D

ABBBBBD
AB-VV-D



And it works for `Sequence` objects, too! Let's switch of the backtracking, though, to save some time.



In [148]:
toolbox.alignment.align(seq, seq2, skip_backtrack=True)

{'distance': 511}

## PDB parser & utilities

Next up is parser for PDB files. The code is just a thin layer over `Bio.PDB`, and we can work directly with its objects, to obtain the whole model...

In [210]:
import toolbox.pdb

bsa = toolbox.pdb.Structure("3v03", join(DATA_FOLDER, "pdb3v03.pdb"))

bsa.structure

/Users/eugen/Library/Caches/pypoetry/virtualenvs/pythonproject-R5KYgqiK-py3.9/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 18563.
  warnings.warn(
/Users/eugen/Library/Caches/pypoetry/virtualenvs/pythonproject-R5KYgqiK-py3.9/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 18566.
  warnings.warn(
/Users/eugen/Library/Caches/pypoetry/virtualenvs/pythonproject-R5KYgqiK-py3.9/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 18573.
  warnings.warn(
/Users/eugen/Library/Caches/pypoetry/virtualenvs/pythonproject-R5KYgqiK-py3.9/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 18596.
  warnings.warn(


<Structure id=3v03>

...or all of the chains...

In [211]:
list(bsa.structure.get_chains())

[<Chain id=A>, <Chain id=B>]

...or all of its residues...

In [151]:
list(bsa.structure.get_residues())[:10] # truncated

[<Residue HIS het=  resseq=3 icode= >,
 <Residue LYS het=  resseq=4 icode= >,
 <Residue SER het=  resseq=5 icode= >,
 <Residue GLU het=  resseq=6 icode= >,
 <Residue ILE het=  resseq=7 icode= >,
 <Residue ALA het=  resseq=8 icode= >,
 <Residue HIS het=  resseq=9 icode= >,
 <Residue ARG het=  resseq=10 icode= >,
 <Residue PHE het=  resseq=11 icode= >,
 <Residue LYS het=  resseq=12 icode= >]

...or finally, all of its atoms.

In [152]:
list(bsa.structure.get_atoms())[:10] # truncated

[<Atom N>,
 <Atom CA>,
 <Atom C>,
 <Atom O>,
 <Atom CB>,
 <Atom N>,
 <Atom CA>,
 <Atom C>,
 <Atom O>,
 <Atom CB>]

We can get a general overview of the structure by looking at its `summary`.

In [153]:
bsa.summary

{'models:': 1, 'chains': 2, 'residues': 1211, 'atoms': 8810}

We can also compute its spatial width.

In [154]:
bsa.width

146.49719889539784

Last but not least, we can show the neighbours of a HETATM ligand in the structure (or any atom at all, in fact). 
Either the neighbouring residues...

In [155]:
atom = bsa.structure[0]['A'][10]['CB']
bsa.search_around(atom, radius=4, level="R")

[<Residue PHE het=  resseq=11 icode= >,
 <Residue ILE het=  resseq=7 icode= >,
 <Residue GLU het=  resseq=6 icode= >,
 <Residue HIS het=  resseq=9 icode= >,
 <Residue ARG het=  resseq=10 icode= >]

...or the neighbouring atoms...

In [156]:
bsa.search_around(atom, radius=4, level="A")

[<Atom NE>,
 <Atom O>,
 <Atom CD>,
 <Atom N>,
 <Atom CA>,
 <Atom CG>,
 <Atom CB>,
 <Atom O>,
 <Atom C>,
 <Atom C>,
 <Atom O>,
 <Atom N>]

...or alteratively neighbouring chains, or models, by passing `"C"` and `"M"` as `level`, respectively. (That doesn't seem particularly useful to me, but it's possile nonetheless.)

Another thing we can do is to calculate the ratios of buried and exposed residues in the structure. The calulcation uses a simple notion of exposure: a residues is considered _exposed_ when any of its atoms is on the surface of the structure.

The method uses `Bio.PDB.ResideDepth` which in turn uses the command-line tool [msms](http://mgltools.scripps.edu/packages/MSMS).

Apart from the ratios themselves, we also get a distribution of the buried and exposed residues, so that we can easily build a histogram.

In [ ]:
bsa.get_residue_exposure()

## Clustal MSA

The next module can be used for simple parsing and scoring of Clustal files. This module is, again, only a thin layer over a class from `Bio`, namely `MultipleSequenceAlignment`. 

We can index a sequence (row) by its position...

In [157]:
import toolbox.msa

msa = toolbox.msa.Clustal(join(DATA_FOLDER, "p53_mafft_clustal.txt"))
msa[0]

SeqRecord(seq=Seq('M-----------------------------------------------RQMESI...DSD'), id='UniRef90_A0A151', name='<unknown name>', description='UniRef90_A0A151', dbxrefs=[])

...or by its id.

In [158]:
msa["UniRef90_A0A151"]

SeqRecord(seq=Seq('M-----------------------------------------------RQMESI...DSD'), id='UniRef90_A0A151', name='<unknown name>', description='UniRef90_A0A151', dbxrefs=[])

We can retrieve whole columns by numpy-style indexing...

In [159]:
msa[:, 130] # 131th column

'--L-EEVV-V--EENSM-W-WWWWWWWWWWWWWWWWWWWWWWWWWFLLWWWCWFW'

...and we can also slice and dice the alignment to our heart's content, to obtain only specific sequences and columns. (Note that this doesn't work with indexing by ID)

In [160]:
msa.alignments[1:10, 2] # 3rd columns from sequences 1, 2, ..., 9

'---------'

Finally, we can use pairwise scoring with a given indel (gap) penalty and a scoring matrix...

In [161]:
from Bio.Align import substitution_matrices
matrix = substitution_matrices.load("BLOSUM62")
matrix

Array([[ 4., -1., -2., -2.,  0., -1., -1.,  0., -2., -1., -1., -1., -1.,
        -2., -1.,  1.,  0., -3., -2.,  0., -2., -1.,  0., -4.],
       [-1.,  5.,  0., -2., -3.,  1.,  0., -2.,  0., -3., -2.,  2., -1.,
        -3., -2., -1., -1., -3., -2., -3., -1.,  0., -1., -4.],
       [-2.,  0.,  6.,  1., -3.,  0.,  0.,  0.,  1., -3., -3.,  0., -2.,
        -3., -2.,  1.,  0., -4., -2., -3.,  3.,  0., -1., -4.],
       [-2., -2.,  1.,  6., -3.,  0.,  2., -1., -1., -3., -4., -1., -3.,
        -3., -1.,  0., -1., -4., -3., -3.,  4.,  1., -1., -4.],
       [ 0., -3., -3., -3.,  9., -3., -4., -3., -3., -1., -1., -3., -1.,
        -2., -3., -1., -1., -2., -2., -1., -3., -3., -2., -4.],
       [-1.,  1.,  0.,  0., -3.,  5.,  2., -2.,  0., -3., -2.,  1.,  0.,
        -3., -1.,  0., -1., -2., -1., -2.,  0.,  3., -1., -4.],
       [-1.,  0.,  0.,  2., -4.,  2.,  5., -2.,  0., -3., -3.,  1., -2.,
        -3., -1.,  0., -1., -3., -2., -2.,  1.,  4., -1., -4.],
       [ 0., -2.,  0., -1., -3., -2., -2.

...to score the whole alignment...

In [162]:
msa.score(matrix, 3)

1993120.0

...or only specific column(s).

In [163]:
msa.score(matrix, 0, cols=[1, 10, 151, 152])

3599.0

Finaly, we can obtain a simple conservation score for a given position (column), which is computed as the ratio of the most common residue in that particular column. So, for column...

In [164]:
len(msa[:, 150]), msa[:, 150]

(55, 'AEGEEG----------S------MMVVE-VK-AEE-A-G----VEQQQE------')

...we can see the most common residue is E with 8 occurences. The score is thus `8 / 55 = 0.145%`. Let's check.

In [165]:
msa.score_conservation(150)

0.14545454545454545

We can also compute the _k_ most conserved positions.

In [174]:
msa.k_most_conserved(5)

[(209, 1.0), (210, 1.0), (213, 1.0), (215, 1.0), (218, 1.0)]